In [ ]:
import boto3
import json
import fitz  # PyMuPDF

# Criar uma sessão com as credenciais fornecidas

# Configurar o cliente da AWS usando a sessão criada
s3_client = session.client('s3')
bedrock_client = session.client('bedrock-runtime', region_name='us-west-2')

def download_pdf_from_s3(bucket_name, object_key, download_path):
    try:
        s3_client.download_file(bucket_name, object_key, download_path)
        print(f"Arquivo {object_key} baixado com sucesso do bucket {bucket_name}.")
    except Exception as e:
        print(f"Erro ao baixar o arquivo do S3: {e}")
        return None

def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        return text.strip()  # Remover espaços em branco no início e no final do texto
    except Exception as e:
        print(f"Erro ao extrair texto do PDF: {e}")
        return None
    

def ask_question_with_pdf(question, pdf_text):
    try:
        corpo = {
            'anthropic_version': 'bedrock-2023-05-31',
            'max_tokens': 300,
            'messages': [
                {
                    'role': 'user',
                    'content': [
                        {
                            'type': 'text',
                            'text': f'Você é um copilot para bancos e fintechs, você deve ajudar a esclarecer dúvidas sobre o mercado financeiro para profissionais que estão chegando agora no mercado ou simplesmente não conhecem alguns termos ou procedimentos da empresa. Você receberá uma dúvida de um usuário e uma documentação para se embasar para responder uma dúvida dele. Caso o documento não possua uma resposta para a pergunta do usuário, use seus conhecimentos gerais para responder. Não fale ao cliente que você está usando documentos para se embasar, você é um copilot. \n Dificuldade do usuário: {question} \n  "Documento para se embasar: {pdf_text}'
                        }
                    ]
                },
            ]
        }
        response = bedrock_client.invoke_model(
            modelId='anthropic.claude-3-sonnet-20240229-v1:0',
            contentType='application/json',
            accept='application/json',
            body=json.dumps(corpo)
        )
        response = json.loads(response['body'].read())
        response_final = response['content'][0]['text']
        return response_final  # Ler e converter a resposta para JSON
    except Exception as e:
        print(f"Erro ao invocar o modelo: {e}")
        return None

def definirDoc(question, filenames):
    try:
        response = bedrock_client.invoke_model(
            modelId='anthropic.claude-3-sonnet-20240229-v1:0',
            contentType='application/json',
            accept='application/json',
            body=json.dumps({
                'anthropic_version': 'bedrock-2023-05-31',
                'max_tokens': 700,
                'messages': [
                    {
                        'role': 'user',
                        'content': [
                            {
                                'type': 'text',
                                'text': f"""Você é auxiliar de sistema da Global Tech Solution que vai filtrar qual é o melhor documento para recomendar para um funcionário. Ele te enviará uma dúvida, você deve ler o nome das documentações existentes e recomendar a melhor para ele. \n Dificuldade do usuário: {question} \n  Documentos disponíveis: {filenames} \n
                                você será utilizada em um sistema, então deve me retornar um json com o nome dos três documentos que você acha que são os melhores para recomendar para o funcionário. siga o modelo: "documento": "nome_do_documento.pdf".
                                não coloque quebras de texto no json.
                                Caso queira recomendar um documento do banco central, coloque "documento": "dadosBacen/" """
                            }
                        ]
                    },

                ]
            })
        )
        response = json.loads(response['body'].read())
        response_final = response['content'][0]['text']
        return response_final   # Ler e converter a resposta para JSON
    except Exception as e:
        print(f"Erro ao invocar o modelo: {e}")
        return None
    
bucket_name = 'chameleon-hackathon-dev'
question = 'COmo ta a implementação do Sistema de Pagamento de Contas Stark Bank aqui na Empresa?'

response = s3_client.list_objects_v2(Bucket=bucket_name)
if 'Contents' in response:
    filenames = [obj['Key'] for obj in response['Contents']]
    print(filenames)
else:
    print("Nenhum arquivo encontrado no bucket.")

object = json.loads(definirDoc(question='Qual é a norma mais recente sobre Tesouro Nacional', filenames=filenames))['documento']
print(object)

if object=='dadosBacen/':
    bucket_name = 'chameleon-hackathon-dev'
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix='dadosBacen/')
    if 'Contents' in response:
        filenames = [obj['Key'] for obj in response['Contents'][1:]]
        print(filenames)

        pdf_texts = []  # Initialize an empty list to store the extracted text from each PDF

        for filename in filenames:
            download_path = filename.split('/')[-1]  # Extract the filename from the full path
            download_pdf_from_s3(bucket_name, filename, download_path)
            pdf_text = extract_text_from_pdf(download_path)
            if pdf_text:
                # Concatenate the extracted text from each PDF
                pdf_texts.append(pdf_text)
            else:
                print(f"Erro ao extrair texto do PDF: {filename}")

        # Print the concatenated text from all the PDFs
        concatenated_text = '\n'.join(pdf_texts)
        print(concatenated_text)
        if pdf_text:
            pdf_texts.append(pdf_text)
        else:
            print(f"Erro ao extrair texto do PDF: {filename}")

        # Print the extracted text from each PDF
        for i, pdf_text in enumerate(pdf_texts):
            textao=f"PDF {i+1}:\n{pdf_text}"

    result = ask_question_with_pdf(question, textao)
    print(result)
    
else:    
    download_pdf_from_s3(bucket_name, object, 'arquivo.pdf')
    texto = extract_text_from_pdf('arquivo.pdf')
    result = ask_question_with_pdf(question, texto)
    print(result)



In [ ]:
def download_pdf_from_s3(bucket_name, object_key, download_path):
    try:
        s3_client.download_file(bucket_name, object_key, download_path)
        print(f"Arquivo {object_key} baixado com sucesso do bucket {bucket_name}.")
    except Exception as e:
        print(f"Erro ao baixar o arquivo do S3: {e}")
        return None

def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        return text.strip()  # Remover espaços em branco no início e no final do texto
    except Exception as e:
        print(f"Erro ao extrair texto do PDF: {e}")
        return None


In [ ]:

def ask_question_with_pdf(question, pdf_text):
    try:
        corpo = {
            'anthropic_version': 'bedrock-2023-05-31',
            'max_tokens': 300,
            'messages': [
                {
                    'role': 'user',
                    'content': [
                        {
                            'type': 'text',
                            'text': f'Você é um copilot para bancos e fintechs, você deve ajudar a esclarecer dúvidas sobre o mercado financeiro para profissionais que estão chegando agora no mercado ou simplesmente não conhecem alguns termos ou procedimentos da empresa. Você receberá uma dúvida de um usuário e uma documentação para se embasar para responder uma dúvida dele. Caso o documento não possua uma resposta para a pergunta do usuário, use seus conhecimentos gerais para responder. Não fale ao cliente que você está usando documentos para se embasar, você é um copilot. \n Dificuldade do usuário: {question} \n  "Documento para se embasar: {pdf_text}'
                        }
                    ]
                },
            ]
        }
        response = bedrock_client.invoke_model(
            modelId='anthropic.claude-3-sonnet-20240229-v1:0',
            contentType='application/json',
            accept='application/json',
            body=json.dumps(corpo)
        )
        response = json.loads(response['body'].read())
        response_final = response['content'][0]['text']
        return response_final  # Ler e converter a resposta para JSON
    except Exception as e:
        print(f"Erro ao invocar o modelo: {e}")
        return None


object_key = object
download_path = 'arquivo.pdf'

if object=='dadosBacen/':
    bucket_name = 'chameleon-hackathon-dev'
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix='dadosBacen/')
    if 'Contents' in response:
        filenames = [obj['Key'] for obj in response['Contents'][1:]]
        print(filenames)

        pdf_texts = []  # Initialize an empty list to store the extracted text from each PDF

        for filename in filenames:
            download_path = filename.split('/')[-1]  # Extract the filename from the full path
            download_pdf_from_s3(bucket_name, filename, download_path)
            pdf_text = extract_text_from_pdf(download_path)
            if pdf_text:
                # Concatenate the extracted text from each PDF
                pdf_texts.append(pdf_text)
            else:
                print(f"Erro ao extrair texto do PDF: {filename}")

        # Print the concatenated text from all the PDFs
        concatenated_text = '\n'.join(pdf_texts)
        print(concatenated_text)
        if pdf_text:
            # Instantiate a single object with the extracted text from each PDF
            # You can use this object for further processing or analysis
            # For example, you can store the text in a list or concatenate them into a single string
            # Here, I'll store the text in a list
            pdf_texts.append(pdf_text)
        else:
            print(f"Erro ao extrair texto do PDF: {filename}")

        # Print the extracted text from each PDF
        for i, pdf_text in enumerate(pdf_texts):
            textao=f"PDF {i+1}:\n{pdf_text}"
    

else:

object = json.loads(definirDoc(question='Qual é a norma mais recente sobre Tesouro Nacional', filenames=filenames))['documento']

# Baixar o PDF do S3
download_pdf_from_s3(bucket_name, object_key, download_path)

# Extrair texto do PDF
pdf_text = extract_text_from_pdf(download_path)

if pdf_text:
    question = 'em qual faculdade o gustavo estuda?'
    result = ask_question_with_pdf(question, pdf_text)
    if result:
        print(result)
    else:
        print("Erro ao obter resposta do modelo.")
else:
    print("Erro ao extrair texto do PDF.")

In [ ]:
import boto3
import json


kendra = session.client('kendra')
bedrock = session.client('bedrock-runtime')

def lambda_handler(event, context):
    query = event['query']
    print(f"Query recebida: {query}")
    
    # Pesquisar no Kendra
    try:
        response = kendra.query(
            IndexId='',
            QueryText=query
        )
        print(f"Resposta do Kendra: {response}")
    except Exception as e:
        print(f"Erro ao consultar o Kendra: {e}")
        return {
            'statusCode': 500,
            'body': f"Erro ao consultar o Kendra: {e}"
        }
    
    docs = response.get('ResultItems')
    print(f"Documentos retornados: {docs}")
    if docs:
        top_doc = docs[0]  # Pegar o documento mais relevante
        content = top_doc['DocumentExcerpt']['Text']
        print(f"Documento encontrado: {content}")
    else:
        content = "Nenhuma informação relevante encontrada."
        try:
            sync_jobs = kendra.list_data_source_sync_jobs(
                IndexId='',
                Id=''  # Substitua pelo ID da sua fonte de dados
            )
            documents_crawled = sync_jobs.get('History', [])
            print(f"Documentos rastreados: {documents_crawled}")
        except Exception as e:
            print(f"Erro ao listar documentos rastreados: {e}")
            documents_crawled = []

        return {
            'statusCode': 200,
            'body': content,
            'documents_crawled': documents_crawled
        }

    # Preparar o corpo da requisição para o Bedrock
    corpo = {
        'anthropic_version': 'bedrock-2023-05-31',
        'max_tokens': 300,
        'messages': [
            {
                'role': 'user',
                'content': [
                    {
                        'type': 'text',
                        'text': content + "\n\n" + query
                    }
                ]
            }
        ]
    }
    
    try:
        # Fazer requisição ao Amazon Bedrock com Claude 3 Sonnet
        response = bedrock.invoke_model(
            modelId='anthropic.claude-3-sonnet-20240229-v1:0',
            contentType='application/json',
            accept='application/json',
            body=json.dumps(corpo).encode('utf-8')
        )
        result = json.loads(response['body'].read())
        generated_text = result['messages'][0]['content'][0]['text']
        print(f"Texto gerado pelo Bedrock: {generated_text}")
        
    except Exception as e:
        print(f"Erro ao invocar o modelo: {e}")
        generated_text = 'Desculpe, não consegui gerar uma resposta.'

    return {
        'statusCode': 200,
        'body': generated_text
    }

lambda_handler(event={"query":"Thiago Volcati"}, context=None)